In [9]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

# text = 'Operating System lies in the category of system software. It basically manages all the resources of the computer. An operating system acts as an interface between the software and different parts of the computer or the computer hardware. The operating system is designed in such a way that it can manage the overall resources and operations of the computer. Operating System is a fully integrated set of specialized programs that handle all the operations of the computer. It controls and monitors the execution of all other programs that reside in the computer, which also includes application programs and other system software of the computer. Examples of Operating Systems are Windows, Linux, Mac OS, etc. An Operating System (OS) is a collection of software that manages computer hardware resources and provides common services for computer programs. The operating system is the most important type of system software in a computer system. The operating system helps in improving the computer software as well as hardware. Without OS, it became very difficult for any application to be user-friendly. The Operating System provides a user with an interface that makes any application attractive and user-friendly. The operating System comes with a large number of device drivers that make OS services reachable to the hardware environment. Each and every application present in the system requires the Operating System. The operating system works as a communication channel between system hardware and system software. The operating system helps an application with the hardware part without knowing about the actual hardware configuration. It is one of the most important parts of the system and hence it is present in every device, whether large or small device. Functions of the Operating System: Resource Management: The operating system manages and allocates memory, CPU time, and other hardware resources among the various programs and processes running on the computer. Process Management: The operating system is responsible for starting, stopping, and managing processes and programs. It also controls the scheduling of processes and allocates resources to them. Memory Management: The operating system manages the computer’s primary memory and provides mechanisms for optimizing memory usage. Security: The operating system provides a secure environment for the user, applications, and data by implementing security policies and mechanisms such as access controls and encryption. Job Accounting: It keeps track of time and resources used by various jobs or users. File Management: The operating system is responsible for organizing and managing the file system, including the creation, deletion, and manipulation of files and directories. Device Management: The operating system manages input/output devices such as printers, keyboards, mice, and displays. It provides the necessary drivers and interfaces to enable communication between the devices and the computer. Networking: The operating system provides networking capabilities such as establishing and managing network connections, handling network protocols, and sharing resources such as printers and files over a network. User Interface: The operating system provides a user interface that enables users to interact with the computer system. This can be a Graphical User Interface (GUI), a Command-Line Interface (CLI), or a combination of both. Backup and Recovery: The operating system provides mechanisms for backing up data and recovering it in case of system failures, errors, or disasters. Virtualization: The operating system provides virtualization capabilities that allow multiple operating systems or applications to run on a single physical machine. This can enable efficient use of resources and flexibility in managing workloads. Performance Monitoring: The operating system provides tools for monitoring and optimizing system performance, including identifying bottlenecks, optimizing resource usage, and analyzing system logs and metrics. Time-Sharing: The operating system enables multiple users to share a computer system and its resources simultaneously by providing time-sharing mechanisms that allocate resources fairly and efficiently. System Calls: The operating system provides a set of system calls that enable applications to interact with the operating system and access its resources. System calls provide a standardized interface between applications and the operating system, enabling portability and compatibility across different hardware and software platforms. Error-detecting Aids: These contain methods that include the production of dumps, traces, error messages, and other debugging and error-detecting methods.'

# Initialise KeyBERT
kw_model = KeyBERT()

# keywords = kw_model.extract_keywords(docs=text, vectorizer=KeyphraseCountVectorizer())

/Users/kaushalpatil/Development/Question Generation Paper/venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline, BartTokenizer, BartForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
def get_question(context, answer, model, tokenizer):
    text = "context: {} answer: {}".format(context, answer)
    encoding = tokenizer.encode_plus(
        text, max_length=384, pad_to_max_length=False, truncation=True, return_tensors="pt"
    ).to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    outs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        max_length=72,
    )
    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    print(dec)
    Question = dec[0].replace("question:", "")
    Question = Question.strip()
    return Question

In [19]:
text = '''
What is Main Memory?
Main memory, commonly referred to as RAM (Random Access Memory), is the computer's primary temporary storage for actively processed data. Unlike permanent storage like hard drives, RAM is volatile, losing its contents when the computer powers down. It's organized into addressable cells, each holding data measured in bytes. Efficient memory management, involving allocation and deallocation, is essential for optimal performance. The amount of RAM directly impacts multitasking and program handling capabilities, making it a critical factor in overall system performance.

What is Memory Management in an Operating System?
Memory management in OS is a technique of controlling and managing the functionality of Random access memory (primary memory). It is used for achieving better concurrency, system performance, and memory utilization.

Memory management moves processes from primary memory to secondary memory and vice versa. It also keeps track of available memory, memory allocation, and unallocated.

Why use Memory Management in OS?
Memory management keeps track of the status of each memory location, whether it is allocated or free.
Memory management enables computer systems to run programs that require more main memory than the amount of free main memory available on the system. This is achieved by moving data between primary and secondary memory.
Memory management addresses the system’s primary memory by providing abstractions such that the programs running on the system perceive a large memory is allocated to them.
It is the job of memory management to protect the memory allocated to all the processes from being corrupted by other processes. If this is not done, the computer may exhibit unexpected/faulty behavior.
Memory management enables sharing of memory spaces among processes, with the help of which, multiple programs can reside at the same memory location (although only one at a time).
Logical Address Space and Physical Address Space
In the realm of memory management, distinguishing between logical address space and physical address space is paramount.

Logical Address Space encompasses the range of addresses that a CPU can generate. This is the perspective from which a program "sees" its memory. For instance, if a computer has 4 GB of RAM, the logical address space might span from 0 to 4 billion. However, it's important to note that this space is not entirely occupied by physical memory. Instead, it serves as a convenient and abstract representation.

Physical Address Space, on the other hand, is the actual physical location in the memory hardware where data is stored. It constitutes the tangible address of a storage cell in RAM. In our previous example, if the RAM modules consist of 4 billion cells, the physical address space would correspond to each of these individual cells.

The Memory Management Unit (MMU) plays a pivotal role in this interplay. It acts as an intermediary, translating logical addresses to physical addresses. This enables programs to operate in a seemingly large logical address space, while efficiently utilizing the available physical memory.

Example: Consider a scenario where a program attempts to access memory address 'x' in its logical address space. The MMU translates this to the corresponding physical address 'y' and retrieves the data from the actual RAM location. This abstraction allows for efficient multitasking and memory allocation

Static and Dynamic Loading
Static Loading involves loading all the necessary program components into the main memory before the program's execution begins. This means that both the executable code and data are loaded into predetermined memory locations. This allocation is fixed and does not change during the program's execution. While it ensures direct access to all required resources, it may lead to inefficiencies in memory usage, especially if the program doesn't utilize all the loaded components.

Dynamic Loading, on the other hand, takes a more flexible approach. In this scheme, a program's components are loaded into the main memory only when they are specifically requested during execution. This results in a more efficient use of memory resources as only the necessary components occupy space. Dynamic loading is particularly advantageous for programs with extensive libraries or functionalities that may not be used in every session.

Static and Dynamic Linking
Static Linking involves incorporating all necessary libraries and modules into the final executable at compile time. This means that the code from libraries is copied into the final executable file. The result is a self-contained executable that doesn't rely on external resources during runtime. While this ensures portability and guarantees that the program will run on any system, it can lead to larger file sizes and potential redundancy if multiple programs use the same libraries.

Dynamic Linking, on the other hand, takes a more dynamic approach. In this method, the necessary libraries are not included in the final executable. Instead, the program dynamically links to the required libraries at runtime. This means that multiple programs can share a single copy of a library, reducing redundancy and conserving memory. However, it does introduce a dependency on the availability of the required libraries at runtime.

Example: Consider a scenario where multiple programs use a common math library. With static linking, each program would contain its own copy of the library, potentially leading to larger file sizes. With dynamic linking, all programs can use the same shared instance of the library, saving disk space

Swapping
Swapping is a technique used in an operating system for efficient management of the memory of the computer system. Swapping includes two tasks, swapping in, and swapping out. Swapping in means placing the blocks or pages of data from the secondary memory to the primary memory. Swapping out is removing blocks/pages of data from the main memory to the Read-Only Memory (R.O.M.). Swapping is useful when a large program has to be executed or some operation has to be performed on a large file.

Swapping

Contiguous Memory Allocation
Contiguous memory allocation is a memory management technique that involves allocating a process to the entire contiguous block of the main memory it requires to execute. This means that the process is loaded into a single continuous chunk of memory. While it's straightforward and efficient in terms of execution, it can lead to issues with fragmentation, where smaller blocks of memory remain unused between allocated processes.

Memory Allocation
Memory allocation is the process of reserving a portion of the computer's memory for a specific application or program. It's a crucial aspect of memory management, ensuring that each running process has enough space to execute efficiently. Effective memory allocation strategies are essential for optimizing system performance.

First Fit

First Fit is a memory allocation algorithm that allocates the first available block of memory that is large enough to accommodate a process. It scans the memory from the beginning and selects the first block that meets the size requirements. While it is relatively simple to implement, it can lead to fragmentation over time.

Best Fit

The Best Fit algorithm searches the entire memory space to find the smallest block that can accommodate a process. This helps in minimizing wastage of memory, as it selects the smallest available block that fits. However, it may lead to more fragmented memory spaces compared to other allocation strategies.

Worst Fit

Worst Fit, as the name implies, allocates the largest available block of memory to a process. This approach can result in more fragmentation compared to First Fit or Best Fit strategies. However, it can be useful in scenarios where larger processes need to be accommodated.

What is Fragmentation?
When processes are moved to and from the main memory, the available free space in primary memory is broken into smaller pieces. This happens when memory cannot be allocated to processes because the size of available memory is less than the amount of memory that the process requires. Such blocks of memory stay unused. This issue is called fragmentation.

Fragmentation is of the following two types:

1. External Fragmentation:
The total amount of free available primary is sufficient to reside a process, but can not be used because it is non-contiguous. External fragmentation can be decreased by compaction or shuffling of data in memory to arrange all free memory blocks together and thus form one larger free memory block.

2. Internal Fragmentation:
Internal fragmentation occurs when the memory block assigned to the process is larger than the amount o memory required by the process. In such a situation a part of memory is left unutilized because it will not be used by any other process. Internal fragmentation can be decreased by assigning the smallest partition of free memory that is large enough for allocating to a process.

fragmentation

What is Paging?
A computer system can address and utilize more memory than the size of the memory present in the computer hardware. This extra memory is referred to as virtual memory. Virtual memory is a part of secondary memory that the computer system uses as primary memory. Paging has a vital role in the implementation of virtual memory.

The process address space is a set of logical addresses that every process refers to in its code. Paging is a technique of memory management in which the process address space is broken into blocks. All the blocks are of the same size and are referred to as “pages”. The size of a page is a power of 2 and its value is in the range of 512 bytes to 8192 bytes. The size of a process is measured in terms of the number of pages.

A similar division of the main memory is done into blocks of fixed size. These blocks are known as “frames” and the size of a frame is the same as that of a page to achieve optimum usage of the primary memory and to avoid external fragmentation.

Following are some Advantages of Paging:
Paging decreases external fragmentation.
Paging is easy to implement.
Paging adds to memory efficiency.
Since the size of the frames is the same as that of pages, - swapping becomes quite simple.
Paging is useful for fast accessing data.
'''

In [20]:
def generate_mcqs(text):
    text = text.strip().replace("\n"," ")
    imp_keywords = kw_model.extract_keywords(docs=text, vectorizer=KeyphraseCountVectorizer())
    id = 0
    questions = []
    print("Key-phrases: ")
    for answer in imp_keywords:
        print(answer[0])
        ques = get_question(text, answer, question_model, question_tokenizer)
        id = len(questions) + 1
        question_data = {
            "ID": id,  # Generate a unique ID
            "Question": ques,
            "answer": answer[0].capitalize(),
        }
        questions.append(question_data)
    return questions

ans = generate_mcqs(text)

Key-phrases: 
memory management
['question: Memory management keeps track of the status of each memory location, whether it is allocated or free.']
memory management technique
['question: Memory management keeps track of the status of each memory location, whether it is allocated or free.']
memory management unit
['question: Memory management keeps track of the status of each memory location, whether it is allocated or free.']
memory management moves processes
['question: Memory management keeps track of the status of each memory location, whether it is allocated or free.']
primary memory
['question: Memory management keeps track of the status of each memory location, whether it is allocated or free.']


In [21]:
import json
with open('result-1.json', 'w') as fp:
    json.dump(ans, fp)

In [22]:
with open('result-1.json', 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent= 4))

[
    {
        "ID": 1,
        "Question": "Memory management keeps track of the status of each memory location, whether it is allocated or free.",
        "answer": "Memory management"
    },
    {
        "ID": 2,
        "Question": "Memory management keeps track of the status of each memory location, whether it is allocated or free.",
        "answer": "Memory management technique"
    },
    {
        "ID": 3,
        "Question": "Memory management keeps track of the status of each memory location, whether it is allocated or free.",
        "answer": "Memory management unit"
    },
    {
        "ID": 4,
        "Question": "Memory management keeps track of the status of each memory location, whether it is allocated or free.",
        "answer": "Memory management moves processes"
    },
    {
        "ID": 5,
        "Question": "Memory management keeps track of the status of each memory location, whether it is allocated or free.",
        "answer": "Primary memory"
    }
]


----

In [23]:
summary_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

def summarizer(text, high = 500, low = 250):
    text = text.strip().replace("\n"," ")
    text = "summarize: "+text
    input_tokens = summary_tokenizer.batch_encode_plus ([text], return_tensors='pt', max_length=1024, truncation=True)['input_ids']
    encoded_ids = summary_model.generate (input_tokens, num_beams=4, length_penalty=2.0, max_length=high, min_length=low, no_repeat_ngram_size=3)
    summary = summary_tokenizer.decode(encoded_ids. squeeze (), skip_special_tokens=True)
    print(summary)
    return summary

In [25]:
def generate_mcqs(text):
    summarized_text = summarizer(text)
    imp_keywords = kw_model.extract_keywords(docs=text, vectorizer=KeyphraseCountVectorizer())
    id = 0
    questions = []
    print("Key-phrases: ")
    for answer in imp_keywords:
        print(answer[0])
        ques = get_question(text, answer, question_model, question_tokenizer)
        id = len(questions) + 1
        question_data = {
            "ID": id,  # Generate a unique ID
            "Question": ques,
            "answer": answer[0].capitalize(),
        }
        questions.append(question_data)
    return questions

ans = generate_mcqs(text)

Main memory, commonly referred to as RAM (Random Access Memory), is the computer's primary temporary storage for actively processed data. Efficient memory management, involving allocation and deallocation, is essential for optimal performance. Memory management enables sharing of memory spaces among processes, with the help of which, multiple programs can reside at the same memory location (although only one at a time) Static and Dynamic Linking Static Linking involves incorporating all necessary libraries and modules into the final executable at compile time. Dynamic Loading, on the other hand, takes a more flexible approach. In this scheme, a program's components are loaded into the main memory only when they are specifically requested during execution. This results in a more efficient use of memory resources as only the necessary components occupy space. It is the job of memory management to protect the memory allocated to all the processes from being corrupted by other processes. I

In [12]:
import json
with open('result-1.1.json', 'w') as fp:
    json.dump(ans, fp)
    
with open('result-1.1.json', 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent= 4))

[
    {
        "ID": 1,
        "Question": "What is the abbreviation for an operating system?",
        "answer": "Operating systems"
    },
    {
        "ID": 2,
        "Question": "What does 'operational system' mean?",
        "answer": "Operating system"
    },
    {
        "ID": 3,
        "Question": "What can enable multiple operating systems to run on a single physical machine?",
        "answer": "Multiple operating systems"
    },
    {
        "ID": 4,
        "Question": "What is the name of the communication channel between system hardware and system software?",
        "answer": "System software"
    },
    {
        "ID": 5,
        "Question": "The Operating System is responsible for organizing and managing what?",
        "answer": "File system"
    }
]
